In [ ]:
import glob
import pathlib
import math

import pandas as pd
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_rows', 500)
import plotly.express as px

from btree_analysis.log_parser import parse_experiment

In [ ]:
EXPERIMENT_PATH = "/tank/btp/testbed/*"
EXPERIMENT_IDENTIFIER = ["payload_size", "flood", "unchanged_counter", "source_id", "iteration"]

In [ ]:
def nextpow2(N):
    """ Function for finding the next power of 2 (see https://gist.github.com/lppier/a59adc18bcf32d8545f7) """
    n = 1
    while n < N: n *= 2
    return n

# Return how long the frame (in bytes) is sent in seconds
def frame_duration(frame_size):
    if pd.isnull(frame_size):
        frame_size = 0
    signal_ext = 6e-6 # For the signal extension (aSignalExtension) of 6 µs in the 2.4 GHz band see IEEE 802.11-2016 10.3.8 and 19.4.4 Table 19-25.
    ofdm_sym_duration = 4e-6 # For 20 MHz channels
    mcs = 7 # Index of used Modulation (see https://wireless.wiki.kernel.org/en/developers/Documentation/ieee80211/802.11n#mcs_rates)
    
    num_fcs_bytes = 4
    num_service_bits = 16
    num_tail_bits = 6
    # See IEEE 802.11-2020 Table 19-27 - 19-30
    mcs_to_num_data_bits_per_symbol  = [26, 52, 78, 104, 156, 208, 234, 260, 52, 104, 156, 208, 312, 416, 468, 520, 78, 156, 234, 312, 468, 624, 702, 780, 104, 208, 312, 416, 624, 832, 936, 1040]
    ndbps = mcs_to_num_data_bits_per_symbol[mcs]
    
    num_psdu_bytes = frame_size + num_fcs_bytes
    num_data_bits = num_service_bits + (8 * num_psdu_bytes) + num_tail_bits
    num_coded_ofdm_syms = math.ceil(num_data_bits / ndbps)
    
    num_ss = math.ceil((mcs + 1) / 8)
    num_phy_header_ofdm_syms = 5 + 2 + 1 + 2^nextpow2(num_ss); # + HT-SIG 2 OFDM syms, HT-STF 1 OFDM sym, HT-LTF 1 OFDM sym * (1, 2, 4, 4)
    
    num_ofdm_syms = num_phy_header_ofdm_syms + num_coded_ofdm_syms
    
    return num_ofdm_syms * ofdm_sym_duration + signal_ext

In [ ]:
events = []

In [ ]:
dirs = glob.glob(EXPERIMENT_PATH)
num_dirs = len(dirs)
for count, experiment in zip(range(0, num_dirs), dirs):
    print(f"Parsing experiment {experiment} ({count + 1}/{num_dirs})")
    _events, _, _ = parse_experiment(pathlib.Path(experiment))
    events += _events

df = pd.DataFrame(events)

In [ ]:
df["mW"] = 10**(df["tx_pwr"]/10)
df["frame_size"] = (df["data_len"] + 34) # Add BTP and Ethernet header
df["mWb"] = df["mW"] * df["frame_size"]
df["mJ"] = df["mW"] * df["frame_size"].apply(frame_duration)

df.loc[(df["event"] == "send") & (df["message"] == "Successfully forwarded payload.") | (df["message"] == "Successfully sent next chunk."), "frame_type"] = "Data"
df.loc[(df["event"] == "send") & ((df["message"] != "Successfully forwarded payload.") & (df["message"] != "Successfully sent next chunk.")), "frame_type"] = "Construction"

In [ ]:
df_tx_0 = df[df["tx_pwr_threshold"] == 0]
df_tx_0 = df_tx_0.drop(["tx_pwr_threshold"], axis=1)

df_tx_0.loc[df_tx_0["flood"] == True, "counter_protocol"] = "SBP"
df_tx_0.loc[df_tx_0["flood"] == False, "counter_protocol"] = df_tx_0["unchanged_counter"]

df_tx_0.loc[df_tx_0["flood"] == True, "protocol"] = "SBP"
df_tx_0.loc[df_tx_0["flood"] == False, "protocol"] = "BTP"

df_tx_0.loc[df_tx_0["payload_size"] == "1K", "payload_size"] = "1 KiB"
df_tx_0.loc[df_tx_0["payload_size"] == "4k", "payload_size"] = "4 KiB"
df_tx_0.loc[df_tx_0["payload_size"] == "16K", "payload_size"] = "16 KiB"

# Absolute Values of Received Payloads

In [ ]:
reception_ratio_df = df_tx_0.groupby(["payload_size", "flood", "counter_protocol", "source_id", "iteration"])["event"].apply(lambda x: (x == 'receive').sum()).reset_index(name='count')
reception_ratio_df.loc[reception_ratio_df["counter_protocol"] == "SBP", "count"] = reception_ratio_df["count"] / 3

In [ ]:
fig = px.box(
    reception_ratio_df,
    x="payload_size",
    y=(reception_ratio_df["count"] / 73) * 100, 
    color="counter_protocol",
    category_orders={
        "payload_size": ["1 KiB", "4 KiB", "16 KiB"]
    },
    labels={
        "payload_size": "Data Size",
        "y": "Successful Recipients (%)",
        "counter_protocol": "Unchanged Counter",
    }
)

fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
)

fig.write_image("/tank/btp/testbed/success_receptions.pdf", width=500, height=220)
fig

In [ ]:
for name, group in reception_ratio_df.groupby(["payload_size", "counter_protocol"]):
    print(f"### {name}")
    print(group.describe())

# [Obsolete] Relative Values of Received Payloads

In [ ]:
def reception_ratio(x):
    flooding = x[x["flood"] == True]
    btp = x[x["flood"] == False]
    
    return float(btp["count"]) / flooding["count"]

reception_count_df = df_tx_0.groupby(EXPERIMENT_IDENTIFIER)["event"].apply(lambda x: (x == 'receive').sum()).reset_index(name='count')
reception_count_df = reception_count_df.groupby(["payload_size", "source_id", "unchanged_counter", "iteration"]).apply(reception_ratio).reset_index(name='ratio')

In [ ]:
fig = px.box(
    reception_count_df,
    x="payload_size",
    y="ratio", 
    color="unchanged_counter",
    category_orders={
        "payload_size": ["1 KiB", "4 KiB", "16 KiB"]
    },
    labels={
        "payload_size": "Data Size",
        "ratio": "Delivery Compared to SBP",
        "unchanged_counter": "Unchanged Counter",
    }
)

fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
)

fig.write_image("/tank/btp/testbed/delivery_ratio.pdf", width=500, height=220)
fig

# Prepare energy df

In [ ]:
def energy_ratio_count(x):
    energy_payload_df = x[x["frame_type"] == "Data"]
    energy_protocol_df = x[x["frame_type"] == "Construction"]
    
    if energy_payload_df.empty:
        energy_payload = 0
    else:
        energy_payload = float(energy_payload_df["mJ_total"])
    
    if energy_protocol_df.empty:
        energy_protocol = 0
    else:
        energy_protocol = float(energy_protocol_df["mJ_total"])
    
    total_energy = energy_payload + energy_protocol
    
    payload_rel = energy_payload / total_energy
    protocol_rel = energy_protocol / total_energy
    
    x.loc[x["frame_type"] == "Data", "mJ_rel"] = payload_rel
    x.loc[x["frame_type"] == "Construction", "mJ_rel"] = protocol_rel
    
    return x

energy_count_df = df_tx_0[df_tx_0["event"] == "send"].groupby(["payload_size", "flood", "counter_protocol", "source_id", "iteration", "frame_type"])["mJ"].sum().reset_index(name='mJ_total')
energy_count_df = energy_count_df.groupby(["payload_size", "flood", "counter_protocol", "source_id", "iteration"]).apply(energy_ratio_count)

energy_count_df.loc[energy_count_df["counter_protocol"] == "SBP", "mJ_total"] = energy_count_df["mJ_total"] / 3

In [ ]:
fig = px.box(
    energy_count_df,
    x="payload_size",
    y=energy_count_df["mJ_total"],
    color="counter_protocol",
    category_orders={
        "payload_size": ["1 KiB", "4 KiB", "16 KiB"]
    },
    labels={
        "payload_size": "Data Size",
        "mJ_total": "Total Energy (mJ)",
        "counter_protocol": "Unchanged Counter",
    }
)

fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
)

fig.write_image("/tank/btp/testbed/total_energy.pdf", width=500, height=220)
fig

In [ ]:
for name, group in energy_count_df.groupby(["payload_size", "counter_protocol"]):
    print(f"### {name}")
    print(group.describe())

# Comparison between flooding and BTP, energy wise

In [ ]:
def energy_ratio_rel(x):
    flooding = x[(x["flood"] == True)].sum()
    btp = x[(x["flood"] == False)].sum()

    return btp["mJ_total"] / flooding["mJ_total"]

energy_ratio_df = df_tx_0[df_tx_0["event"] == "send"].groupby(EXPERIMENT_IDENTIFIER + ["frame_type"])["mJ"].sum().reset_index(name='mJ_total')
energy_ratio_df = energy_ratio_df.groupby(EXPERIMENT_IDENTIFIER).apply(energy_ratio_count)

energy_ratio_df = energy_ratio_df.groupby(["payload_size", "source_id", "unchanged_counter", "iteration"]).apply(energy_ratio_rel).reset_index(name='ratio')

In [ ]:
fig = px.box(
    energy_ratio_df,
    x="payload_size",
    y=(energy_ratio_df["ratio"]) * 100, 
    color="unchanged_counter",
    category_orders={
        "payload_size": ["1 KiB", "4 KiB", "16 KiB"]
    },
    labels={
        "payload_size": "Data Size",
        "y": "Energy Compared to SBP (%)",
        "unchanged_counter": "Unchanged Counter",
    }
)

fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
)

fig.write_image("/tank/btp/testbed/energy_reduction.pdf", width=500, height=220)
fig

# What is BTP's overhead for building the tree

In [ ]:
btp_energy_df = df_tx_0[(df_tx_0["event"] == "send") & (df_tx_0["flood"] == False)].groupby(EXPERIMENT_IDENTIFIER + ["frame_type"])["mJ"].sum().reset_index(name='mJ_total')

In [ ]:
def fill_missing(x):
    if len(x) < 2:
        payload_row = x.iloc[0]
        payload_row["frame_type"] = "Data"
        payload_row["mJ_total"] = 0
        
        new_df = x.append(payload_row, ignore_index=True)
        
        return new_df
    
    else:
        return x



btp_energy_df = btp_energy_df.groupby(EXPERIMENT_IDENTIFIER).apply(fill_missing).reset_index(drop=True)
btp_energy_df = btp_energy_df.groupby(EXPERIMENT_IDENTIFIER).apply(energy_ratio_count)
btp_energy_df = btp_energy_df.groupby(["payload_size", "frame_type"]).mean().reset_index()

In [ ]:
fig = px.bar(
    btp_energy_df,
    x="payload_size",
    y=btp_energy_df["mJ_rel"] * 100,
    color="frame_type",
    category_orders={
        "payload_size": ["1 KiB", "4 KiB", "16 KiB"],
        "frame_type": ["Data", "Construction"]
    },
    labels={
        "payload_size": "Data Size",
        "y": "Payload and Construction Energy (%)",
        "frame_type": "Phase",
    }
)

fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
)

fig.write_image("/tank/btp/testbed/overhead_relative.pdf", width=500, height=220)
fig

In [ ]:
fig = px.bar(
    btp_energy_df,
    x="payload_size",
    y=btp_energy_df["mJ_total"],
    color="frame_type",
    category_orders={
        "payload_size": ["1 KiB", "4 KiB", "16 KiB"],
        "frame_type": ["Data", "Construction"]
    },
    labels={
        "payload_size": "Data Size",
        "mJ_total": "Payload and Construction Energy (mJ)",
        "frame_type": "Phase",
    }
)

fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
)

fig.write_image("/tank/btp/testbed/overhead_absolute.pdf", width=500, height=220)
fig

In [ ]:
fig = px.box(
    df_tx_0[(df_tx_0["event"] == "send") & (df_tx_0["flood"] == False)].groupby(EXPERIMENT_IDENTIFIER + ["frame_type"]).sum().reset_index(),
    x="payload_size",
    y="mJ",
    color="frame_type",
    category_orders={
        "payload_size": ["1 KiB", "4 KiB", "16 KiB"],
        "frame_type": ["Data", "Construction"]
    },
    labels={
        "payload_size": "Data Size",
        "mJ": "Energy Consumption (mJ)",
        "frame_type": "Phase",
    }
)

fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
)

fig.write_image("/tank/btp/testbed/ratio.pdf", width=500, height=220)
fig

In [ ]:
for name, group in df_tx_0[(df_tx_0["event"] == "send") & (df_tx_0["flood"] == False)].groupby(["payload_size", "unchanged_counter", "frame_type"]):
    print(f'### {name}: {group["data_len"].sum() / 1024 / 1024}')

# Strain on Individual Nodes

In [ ]:
def sort_groups(x):
    sorted_x = x.sort_values(by="amount", ascending=False)
    sorted_x['sorted'] = range(len(sorted_x))
    
    return sorted_x

strain_df = df_tx_0.groupby(["payload_size", "protocol", "unchanged_counter", "source_id", "iteration", "node_id"])["mJ"].sum().reset_index(name="amount")
strain_df = strain_df.groupby(["payload_size", "protocol", "unchanged_counter", "source_id", "iteration"]).agg(sort_groups).reset_index(drop=True)
strain_df["normalized"] = (strain_df["amount"] - strain_df["amount"].min()) / (strain_df["amount"].max() - strain_df["amount"].min())

In [ ]:
fig = px.scatter(
    strain_df,
    x="sorted",
    y=strain_df["normalized"] * 100,
    color="protocol",
    facet_row="protocol",
    labels={
        "sorted": "Node",
        "y": "",
        "protocol": "Protocol",
    },
    opacity=.3,
    trendline="lowess",
)

fig.update_traces(
    marker=dict(size=2, opacity=1),
    selector=dict(mode="markers"),
)

fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Protocol=", "")))

fig.update_layout(showlegend=False, margin=dict(l=60, r=10, t=0, b=0))

fig.add_annotation(x=-0.1, y=0.3,
                   text="Load per Node (%)",
                   textangle=-90,
                   xref="paper",
                   yref="paper",
                   font=dict(size=14)
                   )

fig.write_image("/tank/btp/testbed/node_strain.pdf", width=500, height=220)
fig

In [ ]:
def calc_timestamps(x):
    start_events = x[x["event"] == "start"]
    receive_events = x[x["event"] == "receive"]
    finish_events = x[x["message"] == "Successfully sent next chunk."]        
    
    source_node = start_events["source_id"].iloc[0]
    start = start_events[start_events["node_id"] == source_node]["timestamp"].max()
    
    if finish_events.empty:
        last_finish = start
    else:
        last_finish = finish_events["timestamp"].min()
        
    if receive_events.empty:
        last_receive = start
    else:
        last_receive = receive_events["timestamp"].max()

    finish_time = last_finish - start if last_finish >= start else start - start
    receive_time = last_receive - start
    
    x["finish_time"] = finish_time
    x["receive_time"] = receive_time
    
    return x

times_df = df_tx_0[
    (df_tx_0["event"] == "start") |
    (df_tx_0["event"] == "receive") |
    (df_tx_0["message"] == "Successfully sent next chunk.")
].groupby(EXPERIMENT_IDENTIFIER).apply(calc_timestamps)

In [ ]:
fig = px.box(
    times_df,
    x="payload_size",
    y=times_df["finish_time"].dt.total_seconds(),
    color="counter_protocol",
    category_orders={
        "payload_size": ["1 KiB", "4 KiB", "16 KiB"],
        "counter_protocol": [5, 15, 25, "SBP"]
    },
    labels={
        "payload_size": "Data Size",
        "y": "Construction Time (s)",
        "counter_protocol": "Unchanged Counter",
    }
)

fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
)

fig.write_image("/tank/btp/testbed/time_to_build.pdf", width=500, height=220)
fig

In [ ]:
fig = px.box(
    times_df,
    x="payload_size",
    y=times_df["receive_time"].dt.total_seconds(),
    color="counter_protocol",
    category_orders={
        "payload_size": ["1 KiB", "4 KiB", "16 KiB"],
        "counter_protocol": [5, 15, 25, "SBP"]
    },
    labels={
        "payload_size": "Data size",
        "y": "Data Dissemination Time (s)",
        "counter_protocol": "Unchanged Counter",
    }
)

fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
)

fig.write_image("/tank/btp/testbed/time_last_received.pdf", width=500, height=220)
fig